In [1]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
data_dir = r"C:\Users\a051799\OneDrive - Alliance\Documents\Julia_veille_techno\Blog\EuroSAT_RGB\EuroSAT_RGB"
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(64, 64),
  batch_size=32)

Found 27000 files belonging to 10 classes.
Using 21600 files for training.


In [3]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(64, 64),
  batch_size=32)

Found 27000 files belonging to 10 classes.
Using 5400 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [5]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 64, 64, 3)
(32,)


In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
cropper_layer = tf.keras.layers.Cropping2D(cropping=((16, 16)))

cropped_ds = train_ds.map(lambda x, y: (cropper_layer(x), y))
normalized_ds_train = cropped_ds.map(lambda x, y: (normalization_layer(x), y))

cropped_ds = val_ds.map(lambda x, y: (cropper_layer(x), y))
normalized_ds_valid = cropped_ds.map(lambda x, y: (normalization_layer(x), y))

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = normalized_ds_train.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = normalized_ds_valid.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
model = keras.models.Sequential([
    keras.layers.Input((32, 32, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'), # 13 spectral bands
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10) # 10 different classes in the EuroSAT dataset
])

In [10]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

Epoch 1/100


675/675 ━━━━━━━━━━━━━━━━━━━━ 52s 74ms/step - accuracy: 0.3355 - loss: 1.7103 - val_accuracy: 0.5283 - val_loss: 1.2534
Epoch 2/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5769 - loss: 1.1418 - val_accuracy: 0.6920 - val_loss: 0.8665
Epoch 3/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.6829 - loss: 0.8746 - val_accuracy: 0.6933 - val_loss: 0.8250
Epoch 4/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.7153 - loss: 0.7822 - val_accuracy: 0.7320 - val_loss: 0.7549
Epoch 5/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.7351 - loss: 0.7320 - val_accuracy: 0.7476 - val_loss: 0.7129
Epoch 6/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.7489 - loss: 0.6975 - val_accuracy: 0.7541 - val_loss: 0.6912
Epoch 7/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.7554 - loss: 0.6779 - val_accuracy: 0.7593 - val_loss: 0.6923
Epoch 8/100
675/675 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.7610 - loss: 0.6653 - val_accurac

In [11]:
model.save("ground_finder.keras")